1. Web scrapping Wikipedia page for list of postal codes of Canada and stroing the data in a Pandas dataframe

In [2]:
#wiki page link 
link = r"https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 

In [4]:
canada_postcode_wiki = requests.get(link).text
soup = BeautifulSoup(canada_postcode_wiki, 'html5lib')

In [5]:
#Explore Each cell in the table and break down the contents into Postal Code , Borough and Neighborhood
table=soup.find('table')
for row in table.findAll('td'):
    if row.span.text=='Not assigned':
        pass
    else:
        print(row.span.text)
        print(row.p.text)
        print(row.span.text.split("(")[0])
        print(row.span.text.split("(")[1].replace(")",' ').replace('/',',').strip(' '))
        break

North York(Parkwoods)
M3ANorth York(Parkwoods)

North York
Parkwoods


Webscarapping 
Beautiful Soup Code to parse and extract the data from the wiki table and convert it into a pandas data frame

In [6]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        #cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        cell['Neighborhood'] = row.span.text.split("(")[1].replace(")",' ').replace('/',',').strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


Extracted dataframe shape

In [7]:
df.shape

(103, 3)

2. Adding Latitude and logitude data for each Postal Code (Using the geospatial csv file provided in the assessment)

In [11]:
import geocoder

In [13]:
geospatical_file = r"Geospatial_Coordinates.csv"

geospatical_df=pd.read_csv(geospatical_file)
geospatical_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [17]:
#add blank columns for latitude and longitude in main dataframe 
df['Latitude']=''
df['Longitude']=''

#look up geospatcial dataframe and add latitude and longitude into main dataframe 

for i, rowi in df.iterrows():
    for j, rowj in geospatical_df.iterrows():
        if rowj["Postal Code"]== rowi["PostalCode"]:
            rowi["Latitude"]=rowj["Latitude"]
            rowi["Longitude"]=rowj["Longitude"]

df
    

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.65426,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.66586,-79.38316
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509
